In [14]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

from api_keys import g_key


In [15]:
#Create reference to CSV file
csv_path = "CSV/weather_data.csv"

#Impor the CSV into a pandas DataFrame
weather_df = pd.read_csv(csv_path) 
weather_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,paamiut,22,GL,1581116238,77,61.99,-49.67,9.75,13.58
1,1,hobart,75,AU,1581116031,72,-42.88,147.33,61.00,5.82
2,2,norman wells,90,CA,1581116239,76,65.28,-126.83,-5.80,8.05
3,3,cape town,0,ZA,1581116239,68,-33.93,18.42,68.00,9.17
4,4,narsaq,0,GL,1581116239,56,60.92,-46.05,15.80,8.05
...,...,...,...,...,...,...,...,...,...,...
560,560,mildura,0,AU,1581116380,50,-34.20,142.15,81.00,12.75
561,561,antalaha,0,MG,1581116380,73,-14.90,50.28,78.64,10.51
562,562,montbrison,100,FR,1581116381,75,45.67,4.08,42.80,4.70
563,563,dunedin,80,NZ,1581116296,60,-45.87,170.50,52.99,27.22


## Configure Humidity Heatmap

In [16]:
gmaps.configure(api_key=g_key)

figure_layout = {
    'width':'900px',
    'height':'500px',
    'border':'3px solid red',
    'padding':'3px'
}

#Store lat and lng into locations
locations = weather_df[["Lat", "Lng"]].astype(float)

humidity = weather_df["Humidity"]

#Create Heatmap
fig = gmaps.figure(layout=figure_layout)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity=180)

fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(border='3px solid red', height='500px', padding='3px', width='900px'))

In [21]:
#Create DataFrame fitting weather criteria
#Max Temp 70><80
#Wind Speed < 10
#Cloudiness = 0
#Drop rows that do not contain

ideal_weather_df = weather_df[(weather_df["Max Temp"] < 80) & (weather_df["Max Temp"] > 70) & (weather_df["Wind Speed"] < 10) & (weather_df["Cloudiness"] == 0)]

ideal_weather_df.reset_index(drop=True, inplace=True)

In [42]:
ideal_weather_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,dawei,0,MM,1581116240,60,14.08,98.20,76.98,7.54
1,saint-philippe,0,RE,1581116245,69,-21.36,55.77,75.20,6.93
2,saint-pierre,0,RE,1581116247,69,-21.34,55.48,75.20,6.93
3,mumford,0,GH,1581116318,89,5.26,-0.76,76.64,3.69
4,ankpa,0,NG,1581116321,20,7.37,7.63,76.62,2.35
5,brikama,0,GM,1581116323,34,13.27,-16.65,78.80,8.05
6,ban tak,0,TH,1581116332,91,17.04,99.08,73.00,3.49
7,gwadar,0,PK,1581116334,66,25.12,62.33,70.63,8.08
8,dzaoudzi,0,YT,1581116356,94,-12.79,45.27,78.80,3.36
9,amalapuram,0,IN,1581116357,88,16.58,82.02,70.54,3.13


In [43]:
api_key = g_key

#Find Hotels closest to the city
hotels = []

x = 0

for cities in ideal_weather_df["City"]:
    coord = str(ideal_weather_df["Lat"][x]) + ' , ' + str(ideal_weather_df["Lng"][x])
    x+=1
    radius = 5000
    types = 'hotel'
    params = {
        "location": coord,
        "radius": radius,
        "type": types,
        "key": api_key
    }
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    try:
        response = requests.get(url, params=params)
        hotel_results = response.json()
        hotels.append(hotel_results['results'][0]['name'])
        
    except IndexError:
        hotels.append(np.nan)
        
hotels

['Dawei',
 'Saint-Philippe',
 'Saint Pierre',
 'Apam',
 'Ankpa',
 'Brikama',
 'Tak Ok',
 'Gwadar',
 'Mamoudzou',
 'Amalapuram',
 'Saint-Joseph',
 'بير أبو زيد الهلالي',
 'Port Lincoln']

In [44]:
ideal_weather_df["Hotel Name"] = hotels
ideal_weather_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,dawei,0,MM,1581116240,60,14.08,98.20,76.98,7.54,Dawei
1,saint-philippe,0,RE,1581116245,69,-21.36,55.77,75.20,6.93,Saint-Philippe
2,saint-pierre,0,RE,1581116247,69,-21.34,55.48,75.20,6.93,Saint Pierre
3,mumford,0,GH,1581116318,89,5.26,-0.76,76.64,3.69,Apam
4,ankpa,0,NG,1581116321,20,7.37,7.63,76.62,2.35,Ankpa
5,brikama,0,GM,1581116323,34,13.27,-16.65,78.80,8.05,Brikama
6,ban tak,0,TH,1581116332,91,17.04,99.08,73.00,3.49,Tak Ok
7,gwadar,0,PK,1581116334,66,25.12,62.33,70.63,8.08,Gwadar
8,dzaoudzi,0,YT,1581116356,94,-12.79,45.27,78.80,3.36,Mamoudzou
9,amalapuram,0,IN,1581116357,88,16.58,82.02,70.54,3.13,Amalapuram


In [45]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_weather_df.iterrows()]
locations = ideal_weather_df[["Lat", "Lng"]]

In [46]:
#Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='3px solid red', height='500px', padding='3px', width='900px'))